# Minesweeper LLM Competition - Custom GRPO Training

## Goal
Finetune an LLM with LoRA using GRPO to play Minesweeper by:
- **Input**: JSON game state (board configuration)
- **Output**: JSON action (reveal or flag a cell)

Teams will compete to train the best Minesweeper-playing LLM!

## Training Approach
- **Model**: GPT-OSS 20B with LoRA or other models in the /root/.cache/huggingface/hub directory [**Any model other than /root/.cache/huggingface/hub will lead to disqualification**]
- **Method**: GRPO (Group Relative Policy Optimization), SFT or any RL-policies (not just strict to use GRPO)
- **Framework**: Unsloth (2-6x faster, 70% less VRAM)
- **Hardware**: AMD GPU (ROCm)

# Load Model with Unsloth

Load GPT-OSS 20B with LoRA configuration:

In [5]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024
lora_rank = 16

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/root/.cache/huggingface/models--Unsloth--Llama-3.1-8B-Instruct/snapshots/4699cc75b550f9c6f3173fb80f4703b62d946aa5",
    load_in_4bit = True,
    max_seq_length = max_seq_length,
    torch_dtype = torch.bfloat16,
)

print(f"Model device: {model.device}")
print("Model loaded successfully!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
INFO 02-15 03:19:53 [__init__.py:225] Automatically detected platform rocm.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.
==((====))==  Unsloth 2025.10.6: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.11.1rc2.dev161+g8a297115e.rocm700.
   \\   /|    . Num GPUs = 1. Max memory: 255.688 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+gitb2fb688. ROCm Toolkit: 7.0.51831-a3e329ad8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignor

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model device: cuda:0
Model loaded successfully!


# Add LoRA Adapters

Add LoRA layers for efficient finetuning:

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank * 2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

Unsloth 2025.10.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Minesweeper Game Implementation

Custom Minesweeper environment supporting:
- Customizable board size and mine count
- Actions: reveal or flag cells
- Win: reveal all safe cells
- Lose: reveal a mine

In [3]:
from dataclasses import dataclass, field
from typing import List, Tuple, Optional, Set
import random

@dataclass
class MinesweeperGame:
    rows: int
    cols: int
    num_mines: int
    seed: Optional[int] = None
    _rng: random.Random = field(init=False, repr=False)
    _board: List[List[int]] = field(init=False, repr=False)  # -1 = mine, 0-8 = count
    _revealed: Set[Tuple[int, int]] = field(init=False, repr=False, default_factory=set)
    _flagged: Set[Tuple[int, int]] = field(init=False, repr=False, default_factory=set)
    _state: str = field(default="ongoing", init=False, repr=False)

    def __post_init__(self):
        if self.num_mines >= self.rows * self.cols:
            raise ValueError("Too many mines for board size")
        self._rng = random.Random(self.seed)
        self._board = [[0 for _ in range(self.cols)] for _ in range(self.rows)]
        self._place_mines()
        self._calculate_numbers()

    def _place_mines(self):
        """Place mines randomly on the board"""
        positions = [(r, c) for r in range(self.rows) for c in range(self.cols)]
        mine_positions = self._rng.sample(positions, self.num_mines)
        for r, c in mine_positions:
            self._board[r][c] = -1

    def _calculate_numbers(self):
        """Calculate numbers for each cell based on adjacent mines"""
        for r in range(self.rows):
            for c in range(self.cols):
                if self._board[r][c] == -1:
                    continue
                count = 0
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        if dr == 0 and dc == 0:
                            continue
                        nr, nc = r + dr, c + dc
                        if 0 <= nr < self.rows and 0 <= nc < self.cols:
                            if self._board[nr][nc] == -1:
                                count += 1
                self._board[r][c] = count

    def _reveal_cell(self, row: int, col: int) -> bool:
        """Reveal a cell. Returns True if valid move, False if invalid.
        Uses iterative flood-fill to avoid recursion limit on large boards.
        (Issue #11: was recursive; Issue typo: fixed 'bself' -> 'self')
        """
        if not (0 <= row < self.rows and 0 <= col < self.cols):
            return False
        if (row, col) in self._revealed or (row, col) in self._flagged:
            return False

        stack = [(row, col)]
        while stack:
            r, c = stack.pop()
            if (r, c) in self._revealed:
                continue

            self._revealed.add((r, c))

            # Hit a mine!
            if self._board[r][c] == -1:
                self._state = "failed"
                return True

            # Auto-reveal neighbors if cell is 0
            if self._board[r][c] == 0:
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        if dr == 0 and dc == 0:
                            continue
                        nr, nc = r + dr, c + dc
                        if (0 <= nr < self.rows and 0 <= nc < self.cols
                                and (nr, nc) not in self._revealed
                                and (nr, nc) not in self._flagged):
                            stack.append((nr, nc))

        return True

    def _flag_cell(self, row: int, col: int) -> bool:
        """Flag/unflag a cell. Returns True if valid, False if invalid"""
        if not (0 <= row < self.rows and 0 <= col < self.cols):
            return False
        if (row, col) in self._revealed:
            return False
        
        if (row, col) in self._flagged:
            self._flagged.remove((row, col))
        else:
            self._flagged.add((row, col))
        return True

    def do_action(self, action: dict) -> str:
        if self._state != "ongoing":
            return "game_over"
    
        if not isinstance(action, dict):
            self._state = "failed"
            return "invalid_format"
    
        action_type = action.get("type")
        row = action.get("row")
        col = action.get("col")
    
        if action_type not in ["reveal", "flag"] or row is None or col is None:
            self._state = "failed"
            return "invalid_format"
    
        try:
            row, col = int(row), int(col)
        except (ValueError, TypeError):
            self._state = "failed"
            return "invalid_format"
    
        if not (0 <= row < self.rows and 0 <= col < self.cols):
            self._state = "failed"
            return "out_of_bounds"
    
        # ---------------- FIRST MOVE SAFETY ----------------
        if action_type == "reveal" and len(self._revealed) == 0:
            if self._board[row][col] == -1:
                # Move mine somewhere else
                for r in range(self.rows):
                    for c in range(self.cols):
                        if self._board[r][c] != -1:
                            self._board[r][c] = -1
                            self._board[row][col] = 0
                            self._calculate_numbers()
                            break
                    else:
                        continue
                    break
    
        # ---------------- REVEAL ----------------
        if action_type == "reveal":
            if (row, col) in self._revealed:
                return "already_revealed"
            if (row, col) in self._flagged:
                return "flagged_cell"
    
            self._reveal_cell(row, col)
    
        # ---------------- FLAG ----------------
        else:
            if (row, col) in self._revealed:
                return "invalid_flag"
            self._flag_cell(row, col)
    
        self._check_win()
    
        if self._state == "failed":
            return "mine"
        if self._state == "success":
            return "win"
        return "ok"


    def _check_win(self):
        """Check if player has won"""
        total_cells = self.rows * self.cols
        safe_cells = total_cells - self.num_mines
        if len(self._revealed) == safe_cells:
            self._state = "success"

    def get_visible_board(self) -> List[List[str]]:
        """Get board state as player sees it"""
        visible = []
        for r in range(self.rows):
            row = []
            for c in range(self.cols):
                if (r, c) in self._flagged:
                    row.append('F')
                elif (r, c) in self._revealed:
                    val = self._board[r][c]
                    row.append('*' if val == -1 else str(val))
                else:
                    row.append('.')
            visible.append(row)
        return visible

    def state(self) -> str:
        return self._state

    def pretty_print(self) -> str:
        """Pretty print the board"""
        visible = self.get_visible_board()
        lines = []
        
        # Header
        header = "   " + " ".join(f"{i:2d}" for i in range(self.cols))
        lines.append(header)
        lines.append("  " + "─" * (self.cols * 3 + 1))
        
        # Board
        for r, row in enumerate(visible):
            line = f"{r:2d}│ " + "  ".join(row)
            lines.append(line)
        
        return "\n".join(lines)

# Test the Game

In [7]:
# Create test game
game = MinesweeperGame(rows=6, cols=6, num_mines=5)
print(game.pretty_print())
print(f"State: {game.state()}")

# Test action
game.do_action({"type": "reveal", "row": 0, "col": 0})
print("\nAfter revealing (0,0):")
print(game.pretty_print())
print(f"State: {game.state()}")

    0  1  2  3  4  5
  ───────────────────
 0│ .  .  .  .  .  .
 1│ .  .  .  .  .  .
 2│ .  .  .  .  .  .
 3│ .  .  .  .  .  .
 4│ .  .  .  .  .  .
 5│ .  .  .  .  .  .
State: ongoing

After revealing (0,0):
    0  1  2  3  4  5
  ───────────────────
 0│ 0  1  .  .  .  .
 1│ 1  3  .  .  .  .
 2│ .  .  .  .  .  .
 3│ .  .  .  .  .  .
 4│ .  .  .  .  .  .
 5│ .  .  .  .  .  .
State: ongoing


# JSON Input/Output Format

## Input Format (Game State)
```json
{
  "board": [
    ["1", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", "."]
  ],
  "rows": 6,
  "cols": 6,
  "mines": 5,
  "flags_placed": 0,
  "cells_revealed": 0
}
```

## Output Format (Action)
```json
{"type": "reveal", "row": 2, "col": 3}
```
or
```json
{"type": "flag", "row": 1, "col": 4}
```

In [8]:
import json

SYSTEM_PROMPT = "You output JSON actions for Minesweeper. No text, only JSON."

def format_state_for_llm(game: MinesweeperGame) -> str:
    state = {
        "board": game.get_visible_board(),
        "rows": game.rows,
        "cols": game.cols,
        "mines": game.num_mines,
        "flags_placed": len(game._flagged),
        "cells_revealed": len(game._revealed),
    }
    prompt = (
        "You are playing Minesweeper. Analyze the game state and output your next move.\n\n"
        "You must output ONLY a valid JSON object. No explanation, no analysis, no text.\n\n"
        "Just output section after assistantfinal and not anything before it in your output.\n\n"
        "Start your response immediately with { and end with }.\n\n"
        "Do NOT output cell which is already revealed or flagged in the current state.\n\n"
        "Game state:\n"
        f"{json.dumps(state, indent=2)}\n\n"
        "Legend:\n"
        '- "." = unrevealed cell\n'
        '- "F" = flagged cell (suspected mine)\n'
        '- "0"-"8" = number of adjacent mines\n'
        '- "*" = revealed mine (game over)\n\n'
        "Output your next action as JSON:\n"
        '{"type": "reveal", "row": <row_index>, "col": <col_index>}\n'
        "or\n"
        '{"type": "flag", "row": <row_index>, "col": <col_index>}\n\n'
        "Your action:"
    )
    return prompt

def parse_llm_action(response: str) -> dict:
    import re
    best = None
    for match in re.finditer(r'\{[^{}]*\}', response):
        try:
            action = json.loads(match.group())
            if ("type" in action and "row" in action and "col" in action
                    and action["type"] in ["reveal", "flag"]):
                best = action
        except json.JSONDecodeError:
            continue
    return best

# Test
game = MinesweeperGame(rows=6, cols=6, num_mines=5)
prompt = format_state_for_llm(game)
print(prompt[:500] + "...")

You are playing Minesweeper. Analyze the game state and output your next move.

You must output ONLY a valid JSON object. No explanation, no analysis, no text.

Just output section after assistantfinal and not anything before it in your output.

Start your response immediately with { and end with }.

Do NOT output cell which is already revealed or flagged in the current state.

Game state:
{
  "board": [
    [
      ".",
      ".",
      ".",
      ".",
      ".",
      "."
    ],
    [
      "....


In [9]:
# SIMULATE EXACT EVALUATION PIPELINE
# This is EXACTLY what minesweeper_agent.py + minesweeper_model.py will do

import json

# This is what the evaluation server sends as game_state
eval_game_state = {
    "board": [
        ["0", "0", "0", "1", ".", ".", ".", "."],
        ["0", "1", "1", "3", ".", ".", ".", "."],
        ["0", "1", ".", ".", ".", ".", ".", "."],
        ["1", "2", ".", ".", ".", ".", ".", "."],
        [".", ".", ".", ".", ".", "1", "1", "1"],
        [".", ".", ".", ".", ".", "1", "0", "0"],
        [".", ".", ".", ".", ".", "2", "1", "0"],
        [".", ".", ".", ".", ".", ".", "1", "0"]
    ],
    "rows": 8,
    "cols": 8,
    "mines": 10,
    "flags_placed": 0,
    "cells_revealed": 20
}

# EXACT prompt from minesweeper_agent.py build_prompt()
sys_prompt = "You output JSON actions for Minesweeper. No text, only JSON."

eval_prompt = f"""You are playing Minesweeper. Analyze the game state and output your next move.

You must output ONLY a valid JSON object. No explanation, no analysis, no text.

Just output section after assistantfinal and not anything before it in your output.

Start your response immediately with {{ and end with }}.

Do NOT output cell which is already revealed or flagged in the current state.

Game state:
{json.dumps(eval_game_state, indent=2)}

Legend:
- "." = unrevealed cell
- "F" = flagged cell (suspected mine)
- "0"-"8" = number of adjacent mines
- "*" = revealed mine (game over)

Output your next action as JSON:
{{"type": "reveal", "row": <row_index>, "col": <col_index>}}
or
{{"type": "flag", "row": <row_index>, "col": <col_index>}}

Your action:"""

# EXACT model call from minesweeper_model.py generate_response()
text = tokenizer.apply_chat_template(
    [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": eval_prompt},
    ],
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)

generated_ids = model.generate(
    **tokenizer(text, return_tensors="pt").to("cuda"),
    max_new_tokens=128,
)

# EXACT decoding from minesweeper_model.py — skip_special_tokens=True strips <|im_end|>
input_len = tokenizer(text, return_tensors="pt").input_ids.shape[1]
response = tokenizer.decode(generated_ids[0][input_len:], skip_special_tokens=True).strip()

print("=== EXACT EVALUATOR OUTPUT ===")
print(f"Raw response: '{response}'")
print()

# EXACT parsing from minesweeper_agent.py parse_action()
action = parse_llm_action(response)
print(f"Parsed action: {action}")
print(f"Valid: {action is not None}")

=== EXACT EVALUATOR OUTPUT ===
Raw response: 'assistantfinal{"type": "reveal", "row": 4, "col": 0}'

Parsed action: {'type': 'reveal', 'row': 4, 'col': 0}
Valid: True


# Test Model Before Training

See how the base model performs without finetuning:

In [10]:
from transformers import TextStreamer

game = MinesweeperGame(rows=6, cols=6, num_mines=5, seed=42)
prompt = format_state_for_llm(game)

# Removed reasoning_effort="low" — GRPOTrainer does NOT pass it
# during training, so using it only at eval creates a train/eval mismatch.
text = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize = False,
    add_generation_prompt = True,
)

print("=== Base Model Response ===")
output = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 1.0,
    max_new_tokens = 128,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

=== Base Model Response ===
{"type": "reveal", "row": 0, "col": 0}<|eot_id|>


# GRPO Reward Functions

Define reward functions to guide the model's learning:


In [11]:
def ideal_gameplay_scores(completions, **kwargs):
    scores = []
    seeds = kwargs.get("seed", [])
    move_histories = kwargs.get("move_history", [])
    rows_list = kwargs.get("rows", [])
    cols_list = kwargs.get("cols", [])
    mines_list = kwargs.get("num_mines", [])

    for idx, completion in enumerate(completions):
        response = completion[0]["content"]
        action = parse_llm_action(response)

        if action is None or idx >= len(seeds):
            scores.append(-8.0)
            continue

        r, c, m = rows_list[idx], cols_list[idx], mines_list[idx]
        game = MinesweeperGame(rows=r, cols=c, num_mines=m, seed=seeds[idx])

        history = json.loads(move_histories[idx]) if isinstance(move_histories[idx], str) else move_histories[idx]
        for past_action in history:
            game.do_action(past_action)

        try:
            row = int(action.get("row", -1))
            col = int(action.get("col", -1))
            atype = action.get("type", "unknown")
        except:
            scores.append(-8.0)
            continue

        if not (0 <= row < game.rows and 0 <= col < game.cols):
            scores.append(-6.0)
            continue

        score = 0.0

        if atype == "reveal":
            if (row, col) in game._revealed or (row, col) in game._flagged:
                score -= 5.0

            elif game._board[row][col] == -1:
                score -= 10.0

            else:
                if game._board[row][col] > 0:
                    score += 6.0
                else:
                    score += 4.0

                # Proximity bonus only
                near_info = False
                for dr in [-1, 0, 1]:
                    for dc in [-1, 0, 1]:
                        nr, nc = row + dr, col + dc
                        if 0 <= nr < game.rows and 0 <= nc < game.cols:
                            if (nr, nc) in game._revealed:
                                near_info = True
                if near_info:
                    score += 2.0

                if len(game._revealed) + 1 == (game.rows * game.cols - game.num_mines):
                    score += 20.0

        elif atype == "flag":
            if (row, col) in game._revealed or (row, col) in game._flagged:
                score -= 5.0
            elif game._board[row][col] == -1:
                score += 6.0
            else:
                score -= 5.0

        else:
            score -= 6.0

        scores.append(score)

    return scores


# Create Training Dataset

Generate diverse game states for training:

In [ ]:
from datasets import Dataset
import numpy as np
import random
import json

def generate_game_states(num_samples=15000, rng_seed=42):
    """
    Generate Minesweeper training states
    Fixed difficulty: 4x4 board with 2 mines
    Mix of fresh + mid-game states
    """

    np.random.seed(rng_seed)
    random.seed(rng_seed)

    rows, cols, num_mines = 4, 4, 2

    dataset_items = []
    attempts = 0
    max_attempts = num_samples * 10  # increased safety margin

    while len(dataset_items) < num_samples and attempts < max_attempts:
        attempts += 1

        seed = np.random.randint(1_000_000)
        game = MinesweeperGame(rows=rows, cols=cols, num_mines=num_mines, seed=seed)

        # Randomly simulate some moves (0–6 moves)
        num_moves = np.random.randint(0, 7)
        move_history = []

        for _ in range(num_moves):
            if game.state() != "ongoing":
                break

            board = game.get_visible_board()
            unrevealed = [(r, c) for r in range(rows) for c in range(cols) if board[r][c] == '.']

            if not unrevealed:
                break

            r, c = random.choice(unrevealed)

            # 85% reveal, 15% flag
            if random.random() < 0.85:
                action = {"type": "reveal", "row": r, "col": c}
            else:
                action = {"type": "flag", "row": r, "col": c}

            result = game.do_action(action)

            if result in ("mine", "invalid_format", "game_over"):
                break

            move_history.append(action)

        # Only include ongoing states (important)
        if game.state() == "ongoing":
            prompt_text = format_state_for_llm(game)

            dataset_items.append({
                "prompt": [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": prompt_text},
                ],
                "seed": seed,
                "move_history": json.dumps(move_history),
                "rows": rows,
                "cols": cols,
                "num_mines": num_mines,
            })

    print(f"Generated {len(dataset_items)} samples (attempted {attempts})")

    return Dataset.from_list(dataset_items)


# ---- Generate dataset ----

print("Generating training dataset...")
dataset = generate_game_states(num_samples=15000)

print(f"Final dataset size: {len(dataset)}")

from collections import Counter
size_counts = Counter(f"{item['rows']}x{item['cols']}" for item in dataset)
print(f"Board sizes: {dict(size_counts)}")

fresh_count = sum(1 for item in dataset if item["move_history"] == "[]")
print(f"Fresh states: {fresh_count} ({fresh_count/len(dataset)*100:.1f}%)")
print(f"Mid-game states: {len(dataset)-fresh_count} ({(len(dataset)-fresh_count)/len(dataset)*100:.1f}%)")


# Configure GRPO Training

Set up GRPO trainer with all hyperparameters:

In [13]:
from trl import GRPOConfig, GRPOTrainer

# Calculate max lengths
max_prompt_length = 600   # JSON state prompt
max_completion_length = max_seq_length - max_prompt_length

# GRPO Configuration
training_args = GRPOConfig(
    temperature = 0.8,
    learning_rate = 2e-5,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,  # Issue #6: was 1; 4 gives 16 effective completions per update
    num_generations = 16,  # Generate 4 actions per state
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    max_steps = 500,      # Adjust based on compute budget
    save_steps = 200,
    report_to = "none",
    output_dir = "minesweeper_custom_outputs",
)

print("Training configuration:")
print(f"  Max steps: {training_args.max_steps}")
print(f"  Generations per state: {training_args.num_generations}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  LoRA rank: {lora_rank}")

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 16
Training configuration:
  Max steps: 500
  Generations per state: 16
  Learning rate: 2e-05
  LoRA rank: 16


In [14]:
from transformers import TrainerCallback

class MinesweeperEvalCallback(TrainerCallback):
    def __init__(self, eval_every_steps=50, num_games=5):
        self.eval_every_steps = eval_every_steps
        self.num_games = num_games

    def on_step_end(self, args, state, control, model=None, processing_class=None, **kwargs):
        if state.global_step % self.eval_every_steps != 0:
            return
        tokenizer = processing_class
        if tokenizer is None or model is None:
            return
        was_training = model.training
        model.eval()
        wins = 0
        for i in range(self.num_games):
            game = MinesweeperGame(rows=6, cols=6, num_mines=5, seed=10000 + i)
            moves = 0
            while game.state() == "ongoing" and moves < 50:
                prompt = format_state_for_llm(game)
                text = tokenizer.apply_chat_template(
                    [{"role": "system", "content": SYSTEM_PROMPT},
                     {"role": "user", "content": prompt}],
                    tokenize=False, add_generation_prompt=True,
                )
                output = model.generate(
                    **tokenizer(text, return_tensors="pt").to(model.device),
                    temperature=0.7, max_new_tokens=128, do_sample=True,
                )
                response = tokenizer.decode(output[0], skip_special_tokens=True)
                action = parse_llm_action(response)
                if action is None:
                    break
                game.do_action(action)
                moves += 1
            if game.state() == "success":
                wins += 1
        win_rate = wins / self.num_games
        print(f"\n[Eval @ step {state.global_step}] Win rate: {wins}/{self.num_games} ({win_rate*100:.0f}%)\n")
        if was_training:
            model.train()

eval_callback = MinesweeperEvalCallback(eval_every_steps=50, num_games=5)
print("Eval callback created: plays 5 games every 50 steps")

Eval callback created: plays 5 games every 50 steps


# Train the Model

Start GRPO training with reward functions:

In [21]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        ideal_gameplay_scores,
    ],
    args = training_args,
    train_dataset = dataset,
    callbacks = [eval_callback],
)

print("Starting training...")
trainer.train()

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 15,000 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 4 x 1) = 64
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / ideal_gameplay_scores / mean,rewards / ideal_gameplay_scores / std
1,0.000200,2.203125,4.690207,21.031250,19.000000,28.000000,0.000000,21.031250,19.000000,28.000000,0,0,0,0,0,0.043423,2.203125,5.899098
2,0.000100,3.109375,3.964849,20.500000,19.000000,28.000000,0.000000,20.500000,19.000000,28.000000,No Log,No Log,No Log,No Log,No Log,0.033207,3.109375,5.532350
3,0.000100,-0.703125,6.255290,20.406250,19.000000,28.000000,0.000000,20.406250,19.000000,28.000000,No Log,No Log,No Log,No Log,No Log,0.034370,-0.703125,7.841040
4,0.000100,1.859375,3.140350,20.765625,19.000000,28.000000,0.000000,20.765625,19.000000,28.000000,No Log,No Log,No Log,No Log,No Log,0.036554,1.859375,6.125324
5,0.000100,-4.078125,4.805502,20.968750,19.000000,28.000000,0.000000,20.968750,19.000000,28.000000,No Log,No Log,No Log,No Log,No Log,0.028441,-4.078125,5.845375
6,0.000100,-1.140625,3.455838,20.734375,19.000000,28.000000,0.000000,20.734375,19.000000,28.000000,No Log,No Log,No Log,No Log,No Log,0.037048,-1.140625,6.854189
7,0.000100,0.609375,4.959195,20.484375,19.000000,28.000000,0.000000,20.484375,19.000000,28.000000,No Log,No Log,No Log,No Log,No Log,0.028913,0.609375,7.065717
8,0.000100,1.437500,4.871655,20.593750,19.000000,28.000000,0.000000,20.593750,19.000000,28.000000,No Log,No Log,No Log,No Log,No Log,0.034957,1.437500,7.508461
9,0.000100,-0.656250,4.879471,21.203125,19.000000,28.000000,0.000000,21.203125,19.000000,28.000000,No Log,No Log,No Log,No Log,No Log,0.033733,-0.656250,6.799203
10,0.000200,-1.812500,5.858621,20.796875,19.000000,28.000000,0.000000,20.796875,19.000000,28.000000,No Log,No Log,No Log,No Log,No Log,0.039306,-1.812500,6.381708



[Eval @ step 50] Win rate: 0/5 (0%)


[Eval @ step 100] Win rate: 0/5 (0%)


[Eval @ step 150] Win rate: 0/5 (0%)


[Eval @ step 200] Win rate: 0/5 (0%)


[Eval @ step 250] Win rate: 0/5 (0%)


[Eval @ step 300] Win rate: 0/5 (0%)


[Eval @ step 350] Win rate: 0/5 (0%)


[Eval @ step 400] Win rate: 0/5 (0%)


[Eval @ step 450] Win rate: 0/5 (0%)


[Eval @ step 500] Win rate: 0/5 (0%)



TrainOutput(global_step=500, training_loss=0.006460193356935633, metrics={'train_runtime': 4641.4749, 'train_samples_per_second': 6.894, 'train_steps_per_second': 0.108, 'total_flos': 0.0, 'train_loss': 0.006460193356935633})

# Test Trained Model

Evaluate the finetuned model:

In [15]:
# Test on new game
test_game = MinesweeperGame(rows=4, cols=4, num_mines=2)
test_prompt = format_state_for_llm(test_game)

# Removed reasoning_effort="low" for train/eval consistency
test_text = tokenizer.apply_chat_template(
    [{"role": "user", "content": test_prompt}],
    tokenize = False,
    add_generation_prompt = True,
)

print("=== Trained Model Response ===")
output = model.generate(
    **tokenizer(test_text, return_tensors = "pt").to("cuda"),
    temperature = 0.7,
    max_new_tokens = 128,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

# Parse and test action
response_text = tokenizer.decode(output[0])
action = parse_llm_action(response_text)
print(f"\nParsed action: {action}")

if action:
    test_game.do_action(action)
    print(f"\nGame state after action: {test_game.state()}")
    print(test_game.pretty_print())

=== Trained Model Response ===
{"type": "reveal", "row": 0, "col": 0}<|eot_id|>

Parsed action: {'type': 'reveal', 'row': 0, 'col': 0}

Game state after action: ongoing
    0  1  2  3
  ─────────────
 0│ 0  0  1  .
 1│ 1  1  2  .
 2│ .  .  .  .
 3│ .  .  .  .


# Evaluation: Play Complete Games

Test the model on multiple complete games:

In [ ]:
def play_full_game(model, tokenizer, rows=4, cols=4, num_mines=2, seed=None, max_moves=50):
    game = MinesweeperGame(rows=rows, cols=cols, num_mines=num_mines, seed=seed)
    moves = 0
    while game.state() == "ongoing" and moves < max_moves:
        prompt = format_state_for_llm(game)
        text = tokenizer.apply_chat_template(
            [{"role": "system", "content": SYSTEM_PROMPT},
             {"role": "user", "content": prompt}],
            tokenize=False, add_generation_prompt=True,
        )
        output = model.generate(
            **tokenizer(text, return_tensors="pt").to("cuda"),
            temperature=0.3, max_new_tokens=128, do_sample=True,
        )
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        action = parse_llm_action(response)
        if action is None:
            break
        result = game.do_action(action)
        if result == "mine":
            break
        moves += 1
    return game, moves

print("Testing trained model on 20 games...\n")
wins = 0
for i in range(20):
    game, moves = play_full_game(model, tokenizer, seed=i)
    result = game.state()
    if result == "success":
        wins += 1
    tag = "WIN 🏆" if result == "success" else "LOSS"
    print(f"Game {i+1}: {tag} ({result}) after {moves} moves")

print(f"\nWin rate: {wins}/20 ({wins/20*100:.1f}%)")

# Save model
model.save_pretrained("minesweeper_model")
tokenizer.save_pretrained("minesweeper_model")
print("\nModel saved to minesweeper_model/")

# Save the Model

Save your trained model for competition submission:

In [ ]:
def play_full_game(model, tokenizer, rows=6, cols=6, num_mines=5, seed=None, max_moves=50):
    game = MinesweeperGame(rows=rows, cols=cols, num_mines=num_mines, seed=seed)
    moves = 0
    while game.state() == "ongoing" and moves < max_moves:
        prompt = format_state_for_llm(game)
        text = tokenizer.apply_chat_template(
            [{"role": "system", "content": SYSTEM_PROMPT},
             {"role": "user", "content": prompt}],
            tokenize=False, add_generation_prompt=True,
        )
        output = model.generate(
            **tokenizer(text, return_tensors="pt").to("cuda"),
            temperature=0.3, max_new_tokens=128, do_sample=True,
        )
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        action = parse_llm_action(response)
        if action is None:
            break
        result = game.do_action(action)
        if result == "mine":
            break
        moves += 1
    return game, moves

print("Testing trained model on 20 games...\n")
wins = 0
for i in range(20):
    game, moves = play_full_game(model, tokenizer, seed=i)
    result = game.state()
    if result == "success":
        wins += 1
    tag = "WIN 🏆" if result == "success" else "LOSS"
    print(f"Game {i+1}: {tag} ({result}) after {moves} moves")

print(f"\nWin rate: {wins}/20 ({wins/20*100:.1f}%)")

# Save model
model.save_pretrained("minesweeper_model")
tokenizer.save_pretrained("minesweeper_model")
print("\nModel saved to minesweeper_model/")

In [ ]:
# Save LoRA adapters
model.save_pretrained("my_minesweeper_model")
tokenizer.save_pretrained("my_minesweeper_model")

print("Model saved to: my_minesweeper_model/")

# Save merged model in 16bit (local file name which will be used for eval)
if False:
    model.save_pretrained_merged(
        "my_minesweeper_model_merged",
        tokenizer,
        save_method = "merged_16bit"
    )

# Competition Tips

## Improve Your Model:

1. **Adjust Reward Functions**
   - Increase rewards for logical deduction
   - Add penalties for random moves
   - Reward flagging correct mines

2. **Tune Hyperparameters**
   - Increase `max_steps` for longer training
   - Adjust `learning_rate` (try 1e-5 to 1e-4)
   - Increase `lora_rank` for more capacity
   - Adjust `num_generations` (2-8)

3. **Better Training Data**
   - Generate more diverse states
   - Include harder scenarios (more mines)
   - Add states requiring logical deduction

4. **Advanced Techniques**
   - Multi-step rollouts in reward function
   - Curriculum learning (easy → hard boards)
   - Ensemble multiple models

## Team Strategy:
- Experiment with different reward functions
- Try different board sizes during training
- Analyze failed games to improve rewards
- Use temperature sampling during evaluation

Good luck!